The computations in this notebook are based on [this repo](https://github.com/Ghandisanaa/network_datasets).

First we grab some imports.

In [ ]:
import csv
import functools
import itertools
import os

import matplotlib.pyplot as plt
import mpmath
import networkx as nx
import numpy as np

from linear_geodesic_optimization.data import input_network
from linear_geodesic_optimization.data import utility
from linear_geodesic_optimization.plot import get_network_plot


rng = np.random.default_rng()

First we grab an actual network. In this example, we base it off of the toy example from the SIGCOMM paper.

In [ ]:
probes_file_path = os.path.join('..', 'data', 'toy', 'generated', 'probes.csv')
latencies_file_path = os.path.join('..', 'data', 'toy', 'generated', 'connectivity.csv')

graph = input_network.get_graph_from_paths(
    probes_file_path, latencies_file_path,
    epsilon=10.,
    # clustering_distance=500000.,
    ricci_curvature_alpha=0.9999
)

# Store vertex information. Vertices are indices (integers), and labels
# are human-readable strings
n_vertices = len(graph.nodes)
vertex_to_label = list(graph.nodes)
label_to_vertex = {
    label: vertex
    for vertex, label in enumerate(vertex_to_label)
}

# Store edge information. Edges are indices (integers), and labels are
# their corresponding (vertex_source, vertex_destination) pairs. Note
# that each edge is stored twice (once for each orientation)
n_edges = len(graph.edges)
edge_to_label = [
    label
    for i, (u, v) in enumerate(graph.edges)
    for label in [
        (label_to_vertex[u], label_to_vertex[v]),
        (label_to_vertex[v], label_to_vertex[u])
    ]
]
label_to_edge = {
    label: edge
    for edge, label in enumerate(edge_to_label)
}

# Compute the shortest paths. These are stored as a dict of lists of
# edges, where the keys are pairs of vertices
all_pairs_shortest_paths = {
    (label_to_vertex[u], label_to_vertex[v]): [
        (label_to_edge[label_to_vertex[x], label_to_vertex[y]])
        for x, y in itertools.pairwise(shortest_path)
    ]
    for u, shortest_paths in dict(nx.all_pairs_shortest_path(graph)).items()
    for v, shortest_path in shortest_paths.items()
}

In [ ]:
get_network_plot(graph)
plt.show()

In [ ]:
def generate_states_discrete(initial_state, transition_matrix):
    """Run a Markov chain."""
    state = initial_state
    n_states = np.shape(transition_matrix)[0]
    yield state
    while True:
        state = rng.choice(n_states, p=transition_matrix[state, :])
        yield state

def generate_states_continuous(initial_state, q_matrix):
    """Run a Markov chain."""
    state = initial_state
    n_states = np.shape(q_matrix)[0]
    rates = -np.diag(q_matrix)
    stochastic_matrix = q_matrix / rates.reshape((-1, 1))
    np.fill_diagonal(stochastic_matrix, 0.)
    time_remaining = rng.exponential(1 / rates[state])
    while True:
        while time_remaining >= 0.:
            yield state
            time_remaining -= 1.
        state = rng.choice(n_states, p=stochastic_matrix[state, :])
        time_remaining += rng.exponential(1 / rates[state])

def generate_relative_rates(
    horizon, n_nodes, initial_state, q_matrix, rates_per_state
):
    """
    Given a horizon (total number of iterations), the number of nodes to
    get the rates for, the initial state, transition matrix, and the
    rates in a given state, generate a matrix such that:
    * The matrix has `n_nodes` rows and `horizon` columns
    * The maximum of the matrix is 1
    * For a node in the state `s`, the entry is proportional to
      `rates_per_state[s]`
    """
    # First, generate the non-normalized matrix of rates
    rates = np.zeros((n_nodes, horizon))
    for node in range(n_nodes):
        rates[node, :] = np.array([
            rates_per_state[state]
            for state in itertools.islice(
                generate_states_continuous(initial_state, q_matrix),
                horizon
            )
        ])

    # Now normalize
    return rates / np.amax(rates)

def generate_relative_rates_pathwise(
    horizon, n_nodes, edges, shortest_paths, initial_state, q_matrix, rates_per_state
):
    rates = np.zeros((n_nodes, horizon))
    for s in range(n_nodes):
        for d in range(n_nodes):
            if s == d:
                continue

            # Amount of traffic from the route from s to d
            route_rates = np.array([
                rates_per_state[state]
                for state in itertools.islice(
                    generate_states_continuous(initial_state, q_matrix),
                    horizon
                )
            ])

            for e in shortest_paths[(s, d)]:
                s_, d_ = edges[e]
                rates[s_] += route_rates
                rates[d_] += route_rates

    return rates / np.amax(rates)

In [ ]:
days = 2
horizon = days * 24 + 1  # Number of measurements we're taking
t = np.linspace(0., days, horizon)
q_matrix = np.array([
    [-0.05, 0.05],
    [0.2, -0.2]
])

rates_source = generate_relative_rates(horizon, n_vertices, 0, q_matrix, [1./3., 3./4.])
rates_destination = generate_relative_rates(horizon, n_vertices, 0, q_matrix, [1./3., 3./4.])

# rates_source = rates_destination = generate_relative_rates_pathwise(
#     horizon, n_vertices, edge_to_label, all_pairs_shortest_paths, 0, q_matrix, [1./3., 3./4.]
# )

In [ ]:
elliptic_theta = np.vectorize(mpmath.jtheta, 'D')

def get_volume(t):
    """
    Generate the total amount of traffic in a system at a time of day
    ranging from 0 to 1. The output also lies in [0, 1].
    """
    mus = [9./24., 14./24., 19./24.]
    sigmas = [0.1, 0.1, 0.1]
    # TODO: Add some random noise
    volume = np.real(1. + sum([
        np.sqrt(np.pi) * sigma * elliptic_theta(3, np.pi * (mu - t), np.exp(-(np.pi * sigma)**2))
        for mu, sigma in zip(mus, sigmas)
    ]) / len(mus))
    return volume / np.amax(volume)

In [ ]:
volume = get_volume(t)
plt.plot(t, volume)
plt.ylim(bottom=0.)
plt.show()

In [ ]:
# At this point, in a 2 day period, traffic on a link from s to d at
# time t is given by
# rates_source[s, t] * rates_destination[d, t] * volume[t]

def estimate_delays_single_link(s, d, rates_source, rates_destination, volume, mu=1., c=1.):
    """
    Estimate delay given traffic.

    `mu` is the delay of the link under no load. `c` is a parameter
    controlling how much load should affect delay (higher = less
    effect).
    """
    return mu / (1 - rates_source[s, :] * rates_destination[d, :] * volume / c)

def estimate_delays(s, d, all_pairs_shortest_paths, delays_single_link):
    return sum(
        [
            delays_single_link[edge]
            for edge in all_pairs_shortest_paths[(s, d)]
        ],
        np.zeros(delays_single_link[0].shape)
    ) + sum(
        [
            delays_single_link[edge]
            for edge in all_pairs_shortest_paths[(d, s)]
        ],
        np.zeros(delays_single_link[0].shape)
    )

In [ ]:
estimated_delays_single_link = [
    estimate_delays_single_link(
        s, d,
        rates_source, rates_destination,
        volume,
        utility.get_GCL(
            (v_s['lat'], v_s['long']),
            (v_d['lat'], v_d['long'])
        ),
        5.
    )
    for (s, d) in edge_to_label
    for v_s in (graph.nodes[vertex_to_label[s]],)
    for v_d in (graph.nodes[vertex_to_label[d]],)
]
estimated_delays = {
    (s, d): estimate_delays(s, d, all_pairs_shortest_paths, estimated_delays_single_link)
    for s in range(n_vertices)
    for d in range(n_vertices)
}

In [ ]:
for (s, d), estimated_delays_vector in estimated_delays.items():
    if s == d:
        continue
    v_s = graph.nodes[vertex_to_label[s]]
    v_d = graph.nodes[vertex_to_label[d]]
    gcl = utility.get_GCL(
        (v_s['lat'], v_s['long']),
        (v_d['lat'], v_d['long'])
    )
    plt.plot(estimated_delays_vector - gcl)
plt.title('Residual Latencies vs. Time')
plt.ylabel('Residual Latency')
plt.xlabel('Time')
plt.ylim(bottom=0.)
plt.show()

In [ ]:
for i in range(horizon):
    with open(os.path.join('..', 'data', 'toy', 'generated', 'latencies', f'{i}.csv'), 'w') as f:
        writer = csv.DictWriter(f, ['source_id', 'target_id', 'rtt'])
        writer.writeheader()
        for (s, d), delays in estimated_delays.items():
            writer.writerow({
                'source_id': vertex_to_label[s],
                'target_id': vertex_to_label[d],
                'rtt': delays[i],
                # 'rtt': min(estimated_delays[(s, d)][i], estimated_delays[(d, s)][i])
            })